# Data Preprocessing Tools

## Importing the libraries

In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [21]:
# Load Excel files
df_s4 = pd.read_excel("0_am_workday_raw_data/resources_s4.xlsx")
df_workday = pd.read_excel("0_am_workday_raw_data/resources_workday.xlsx")

In [22]:
print(df_s4.columns)
print(df_workday.columns)

Index(['SAP Personnel #', 'Company Code', 'Hire_Date', 'Current_Role',
       'Last_Name', 'First_Name', 'Cost Center', 'SSN\n (REQ FIELD)',
       'Birthdate', 'Street Address', 'City', 'State', 'Zip Code'],
      dtype='object')
Index(['Employee_ID', 'Employee Annual Salary$', 'TAXES$',
       'Monthly Medical contribution$', 'Monthly Dental Contribution$',
       'Monthly Vision Contribution$', 'Bonus $',
       'OVERHEAD (Laptop; Desk; Supplies)$', 'Years_Of_Service',
       'Last_Promoted', 'Department', 'Location', 'Manager', 'Gender',
       'Employee HR rate', '# of Hours per week',
       'Calculated Column - Fully Loaded Cost'],
      dtype='object')


In [23]:
# Selective renaming for df_s4
df_s4_rename = df_s4.rename(columns={
    'SAP Personnel #': 'SAP_Personnel',
    'Zip Code': 'Zip_Code',
    'Company Code':'Company_Code',
    'SSN\n (REQ FIELD)':'SSN',
    'Street Address':'Street_Address'
})

# Selective renaming for df_workday
df_workday_rename = df_workday.rename(columns={
    'Employee Annual Salary$': 'Employee_Annual_Salary',
    'Monthly Medical contribution$': 'Monthly_Medical_contribution',
    'Monthly Dental Contribution$': 'Monthly_Dental_Contribution',
    'Monthly Vision Contribution$': 'Monthly_Vision_Contribution',
    'Bonus $': 'Bonus',
    'Employee HR rate': 'Employee_HR_rate',
    '# of Hours per week': 'Hours_per_week',
    'TAXES$':'TAXES',
    'OVERHEAD (Laptop; Desk; Supplies)$':'OVERHEAD',
    'Calculated Column - Fully Loaded Cost':'Fully_Loaded_Cost'
})


In [24]:
print(df_s4_rename.columns)
print(df_workday_rename.columns)

Index(['SAP_Personnel', 'Company_Code', 'Hire_Date', 'Current_Role',
       'Last_Name', 'First_Name', 'Cost Center', 'SSN', 'Birthdate',
       'Street_Address', 'City', 'State', 'Zip_Code'],
      dtype='object')
Index(['Employee_ID', 'Employee_Annual_Salary', 'TAXES',
       'Monthly_Medical_contribution', 'Monthly_Dental_Contribution',
       'Monthly_Vision_Contribution', 'Bonus', 'OVERHEAD', 'Years_Of_Service',
       'Last_Promoted', 'Department', 'Location', 'Manager', 'Gender',
       'Employee_HR_rate', 'Hours_per_week', 'Fully_Loaded_Cost'],
      dtype='object')


In [25]:
# Your selected columns
s4_req_cols = ['SAP_Personnel', 'Current_Role', 'Birthdate']
workday_req_cols = ['Employee_ID', 'Years_Of_Service', 'Last_Promoted', 'Department',
                    'Employee_HR_rate', 'Hours_per_week', 'Fully_Loaded_Cost']

# Load only required columns
df_s4 = df_s4_rename[s4_req_cols]
df_workday = df_workday_rename[workday_req_cols]


In [26]:
print(df_s4.head())
print(df_workday.head())

   SAP_Personnel               Current_Role  Birthdate
0             57   Senior Software Engineer 1991-05-19
1             58   Associate Data Scientist 1996-03-23
2             59  Associate Product Manager 1993-09-25
3             60           Business Analyst 1994-03-03
4             61    Chief Operating Officer 1984-01-13
   Employee_ID  Years_Of_Service Last_Promoted    Department  \
0           57               5.2    2022-08-15   Engineering   
1           58               1.7    2023-11-10  Data Science   
2           59               1.3    2024-03-05       Product   
3           60               3.9    2023-02-15       Finance   
4           61              15.2    2019-08-22     Executive   

   Employee_HR_rate  Hours_per_week  Fully_Loaded_Cost  
0                60              40          163355.20  
1               117              40           92926.13  
2               147              40          160081.53  
3               153              40          105775.38  


## Merging the dataset

In [27]:
# Merge on SAP Personnel # and Employee_ID
df_merged = pd.merge(df_s4, df_workday, left_on='SAP_Personnel', right_on='Employee_ID', how='inner')

# Drop both ID columns since they are just identifiers
df_merged.drop(columns=['SAP_Personnel'], errors='ignore', inplace=True)

In [28]:
print(df_merged.head())

                Current_Role  Birthdate  Employee_ID  Years_Of_Service  \
0   Senior Software Engineer 1991-05-19           57               5.2   
1   Associate Data Scientist 1996-03-23           58               1.7   
2  Associate Product Manager 1993-09-25           59               1.3   
3           Business Analyst 1994-03-03           60               3.9   
4    Chief Operating Officer 1984-01-13           61              15.2   

  Last_Promoted    Department  Employee_HR_rate  Hours_per_week  \
0    2022-08-15   Engineering                60              40   
1    2023-11-10  Data Science               117              40   
2    2024-03-05       Product               147              40   
3    2023-02-15       Finance               153              40   
4    2019-08-22     Executive                79              40   

   Fully_Loaded_Cost  
0          163355.20  
1           92926.13  
2          160081.53  
3          105775.38  
4          412283.65  


In [29]:
print(df_merged.shape)

(100, 9)


## Feature Engineering

In [30]:
df_merged['Last_Promoted'] = pd.to_datetime(df_merged['Last_Promoted'])
df_merged['Birthdate'] = pd.to_datetime(df_merged['Birthdate'])
df_merged['Monthly_FLC'] = df_merged['Fully_Loaded_Cost']/12

today = pd.Timestamp.today()

# Calculate years since last promotion using days / 365.25
df_merged['Years_Since_Last_Promotion'] = (today - df_merged['Last_Promoted']).dt.days / 365.25

# Calculate age as integer years
df_merged['age'] = ((today - df_merged['Birthdate']).dt.days / 365.25).astype(int)


# Renaming DataFrame for feature-engineered data
df_featured = df_merged.copy()

# Optionally drop original columns
df_featured.drop(columns=['Last_Promoted', 'Birthdate'], inplace=True)

print(df_featured[['Years_Since_Last_Promotion', 'age']].head())

   Years_Since_Last_Promotion  age
0                    2.814511   34
1                    1.577002   29
2                    1.259411   31
3                    2.310746   31
4                    5.796030   41


In [31]:
print(df_featured.head())

                Current_Role  Employee_ID  Years_Of_Service    Department  \
0   Senior Software Engineer           57               5.2   Engineering   
1   Associate Data Scientist           58               1.7  Data Science   
2  Associate Product Manager           59               1.3       Product   
3           Business Analyst           60               3.9       Finance   
4    Chief Operating Officer           61              15.2     Executive   

   Employee_HR_rate  Hours_per_week  Fully_Loaded_Cost   Monthly_FLC  \
0                60              40          163355.20  13612.933333   
1               117              40           92926.13   7743.844167   
2               147              40          160081.53  13340.127500   
3               153              40          105775.38   8814.615000   
4                79              40          412283.65  34356.970833   

   Years_Since_Last_Promotion  age  
0                    2.814511   34  
1                    1.577002 

In [32]:
print(df_featured.columns)

Index(['Current_Role', 'Employee_ID', 'Years_Of_Service', 'Department',
       'Employee_HR_rate', 'Hours_per_week', 'Fully_Loaded_Cost',
       'Monthly_FLC', 'Years_Since_Last_Promotion', 'age'],
      dtype='object')


In [33]:
print(df_featured.shape)

(100, 10)


## Create Custom Dependent Column

In [34]:
# Create a boolean Series for condition
condition = df_featured['Years_Since_Last_Promotion'] > 5

# Group by that boolean condition
grouped = df_featured.groupby(condition)


In [35]:
print(grouped.size())

Years_Since_Last_Promotion
False    89
True     11
dtype: int64


In [36]:
df_featured['left'] = (df_featured['Years_Since_Last_Promotion'] > 5).astype(int)

# Renaming DataFrame for updated-engineered data
df_updated = df_featured.copy()

In [37]:
print(df_updated[['Years_Since_Last_Promotion', 'left']].head())

   Years_Since_Last_Promotion  left
0                    2.814511     0
1                    1.577002     0
2                    1.259411     0
3                    2.310746     0
4                    5.796030     1


## Export Updated Excel

In [38]:
# Create the folder if it doesn't exist
os.makedirs("1_am_workday_updated_data", exist_ok=True)

# Save the Excel file inside the folder
df_updated.to_csv("1_am_workday_updated_data/1_am_workday_updated_data.csv", index=False)
